In [11]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm 
import dtale
import ScraperFC as sfc

In [2]:
fbref_scraper = sfc.FBref()
df_pl_games_dl = fbref_scraper.scrape_matches('2024-2025', 'EPL')
df_pl_games_dl.to_pickle('../data/epl_2024_2025.pkl')

2024-2025 EPL matches: 100%|██████████| 110/110 [14:10<00:00,  7.73s/it]


In [221]:
def read_pickle(file_path: str) -> pd.DataFrame:
    '''
        This function reads a csv file and returns a DataFrame.
        
        Parameters:
            - file_path (str): Path to the csv file
            
        Returns:
            - df (pd.DataFrame): DataFrame of the csv file
    '''
    
    df_games = pd.read_pickle(file_path)
    
    df_games['league'] = df_games['Stage'].apply(lambda x: x.split('(')[0])
    df_games['matchweek'] = df_games['Stage'].apply(lambda x: int(x.split('(')[1].split(' ')[1].replace(')', '')))
    df_games.drop(columns=['Stage'], inplace=True)
    
    df_games.columns = df_games.columns.str.strip().str.lower().str.replace(' ', '_')
    
    return df_games

def construct_teams_series(df_games: pd.DataFrame) -> pd.DataFrame:
    '''
       This function takes a DataFrame of games and returns a DataFrame of teams with their form up to a certain matchweek.
              
       Parameters:
              - df_games (pd.DataFrame): DataFrame of games
              - matchweek (int): Matchweek to consider
                     
       Returns:
              - df_clubs_series (pd.DataFrame): DataFrame of teams with their form up to a certain matchweek
    '''
    
    df_melted = df_games.melt(id_vars=['home_team_id', 'away_team_id', 'home_goals', 'home_xg', 'away_goals', 'away_xg', 'league', 'matchweek'], value_vars=['home_team', 'away_team'],
                    var_name='location', value_name='team')
    df_melted['team_id'] = df_melted.apply(lambda x: x['home_team_id'] if x['location'] == 'home_team' else x['away_team_id'], axis=1)
    df_melted.drop(columns=['home_team_id', 'away_team_id'], inplace=True)

    df_grouped = df_melted.groupby('team').apply(lambda x: x, include_groups=False)
    df_grouped.sort_values(by=['team_id', 'team', 'matchweek'], inplace=True)

    df_grouped['location_table'] = df_grouped['location'].apply(lambda x: 'H' if x == 'home_team' else 'A')

    df_grouped['form'] = [  'W' if (x > y and l == 'home_team') or 
                                    (x < y and l == 'away_team') else
                            'D' if x == y else 
                            'L' for x, y, l in zip(df_grouped['home_goals'], df_grouped['away_goals'], df_grouped['location'])]

    df_grouped['home_form'] = [ '' if l == 'away_team' else
                                'W' if x > y else
                                'D' if x == y else 
                                'L' for x, y, l in zip(df_grouped['home_goals'], df_grouped['away_goals'], df_grouped['location'])]

    df_grouped['away_form'] = [ '' if l == 'home_team' else
                                'W' if x < y else
                                'D' if x == y else 
                                'L' for x, y, l in zip(df_grouped['home_goals'], df_grouped['away_goals'], df_grouped['location'])]
    
    df_grouped['expected_form'] = [ 'W' if (x > y and l == 'home_team') or 
                                    (x < y and l == 'away_team') else
                                    'D' if x == y else 
                                    'L' for x, y, l in zip(df_grouped['home_xg'], df_grouped['away_xg'], df_grouped['location'])]

    df_grouped['expected_home_form'] = [ '' if l == 'away_team' else
                                'W' if x > y else
                                'D' if x == y else 
                                'L' for x, y, l in zip(df_grouped['home_xg'], df_grouped['away_xg'], df_grouped['location'])]

    df_grouped['expected_away_form'] = [ '' if l == 'home_team' else
                                'W' if x < y else
                                'D' if x == y else 
                                'L' for x, y, l in zip(df_grouped['home_xg'], df_grouped['away_xg'], df_grouped['location'])]

    df_grouped['goals_form'] = [str(x) if l == 'home_team' else str(y) for x, y, l in zip(df_grouped['home_goals'], df_grouped['away_goals'], df_grouped['location'])]
    df_grouped['home_goals_form'] = [str(x) if l == 'home_team' else '' for x, l in zip(df_grouped['home_goals'], df_grouped['location'])]
    df_grouped['away_goals_form'] = ['' if l == 'home_team' else str(y) for y, l in zip(df_grouped['away_goals'], df_grouped['location'])]

    df_grouped['goals_form_against'] = [str(x) if l == 'away_team' else str(y) for x, y, l in zip(df_grouped['home_goals'], df_grouped['away_goals'], df_grouped['location'])]
    df_grouped['home_goals_form_against'] = [str(x) if l == 'home_team' else '' for x, l in zip(df_grouped['away_goals'], df_grouped['location'])]
    df_grouped['away_goals_form_against'] = [str(x) if l == 'away_team' else '' for x, l in zip(df_grouped['home_goals'], df_grouped['location'])]

    df_grouped['xg_form'] = [str(x) if l == 'home_team' else str(y) for x, y, l in zip(df_grouped['home_xg'], df_grouped['away_xg'], df_grouped['location'])]
    df_grouped['home_xg_form'] = [str(x) if l == 'home_team' else '' for x, l in zip(df_grouped['home_xg'], df_grouped['location'])]
    df_grouped['away_xg_form'] = ['' if l == 'home_team' else str(y) for y, l in zip(df_grouped['away_xg'], df_grouped['location'])]

    df_grouped['xg_form_against'] = [str(x) if l == 'away_team' else str(y) for x, y, l in zip(df_grouped['home_xg'], df_grouped['away_xg'], df_grouped['location'])]
    df_grouped['home_xg_form_against'] = [str(x) if l == 'home_team' else '' for x, l in zip(df_grouped['away_xg'], df_grouped['location'])]
    df_grouped['away_xg_form_against'] = [str(x) if l == 'away_team' else '' for x, l in zip(df_grouped['home_xg'], df_grouped['location'])]

    df_clubs_series = df_grouped.groupby(['team_id', 'team']).agg({ 'location_table': lambda x: ''.join(x),
                                                                    'form': lambda x: ''.join(x),
                                                                    'home_form': lambda x: ''.join(x),
                                                                    'away_form': lambda x: ''.join(x),
                                                                    'expected_form': lambda x: ''.join(x),
                                                                    'expected_home_form': lambda x: ''.join(x),
                                                                    'expected_away_form': lambda x: ''.join(x),
                                                                    'goals_form': lambda x: '-'.join(x),
                                                                    'home_goals_form': lambda x: '-'.join(filter(None, x)),
                                                                    'away_goals_form': lambda x: '-'.join(filter(None, x)),
                                                                    'goals_form_against': lambda x: '-'.join(x),
                                                                    'home_goals_form_against': lambda x: '-'.join(filter(None, x)),
                                                                    'away_goals_form_against': lambda x: '-'.join(filter(None, x)),
                                                                    'xg_form': lambda x: '-'.join(x),
                                                                    'home_xg_form': lambda x: '-'.join(filter(None, x)),
                                                                    'away_xg_form': lambda x: '-'.join(filter(None, x)),
                                                                    'xg_form_against': lambda x: '-'.join(x),
                                                                    'home_xg_form_against': lambda x: '-'.join(filter(None, x)),
                                                                    'away_xg_form_against': lambda x: '-'.join(filter(None, x))
                                                                }).reset_index()
        
    return df_clubs_series


def get_table_pos(df_teams_series: pd.DataFrame, matchweek: int, team_id: str) -> pd.DataFrame:
        '''
                This function takes a DataFrame of teams with their form and a matchweek and returns the league position of a team at that matchweek.
                
                Parameters:
                - df_teams_series (pd.DataFrame): DataFrame of teams with their form
                - matchweek (int): Matchweek to consider
                
                Returns:
                - pos (pd.DataFrame): League position of a team at that matchweek
        '''
        
        df_teams_series = df_teams_series.copy()
        
        #Total
        df_teams_series['form'] = df_teams_series['form'].str[0:matchweek-1]
        df_teams_series['expected_form'] = df_teams_series['expected_form'].str[0:matchweek-1]

        df_teams_series['total_points'] = df_teams_series['form'].apply(lambda x: sum([3 if res == 'W' else 1 if res == 'D' else 0 for res in x]))
        df_teams_series['expected_total_points'] = df_teams_series['expected_form'].apply(lambda x: sum([3 if res == 'W' else 1 if res == 'D' else 0 for res in x]))
        df_teams_series['expected_points_diff'] = df_teams_series['total_points'] - df_teams_series['expected_total_points']
        df_teams_series['league_pos'] = df_teams_series['total_points'].rank(ascending=False, method='min')
        
        #Home
        df_teams_series['home_form'] = df_teams_series['home_form'].str[0:matchweek-1]
        df_teams_series['expected_home_form'] = df_teams_series['expected_home_form'].str[0:matchweek-1]

        df_teams_series['home_total_points'] = df_teams_series['home_form'].apply(lambda x: sum([3 if res == 'W' else 1 if res == 'D' else 0 for res in x]))
        df_teams_series['home_expected_total_points'] = df_teams_series['expected_home_form'].apply(lambda x: sum([3 if res == 'W' else 1 if res == 'D' else 0 for res in x]))
        df_teams_series['home_expected_points_diff'] = df_teams_series['home_total_points'] - df_teams_series['home_expected_total_points']
        df_teams_series['home_league_pos'] = df_teams_series['home_total_points'].rank(ascending=False, method='min')
        
        #Away
        df_teams_series['away_form'] = df_teams_series['away_form'].str[0:matchweek-1]
        df_teams_series['expected_away_form'] = df_teams_series['expected_away_form'].str[0:matchweek-1]

        df_teams_series['away_total_points'] = df_teams_series['away_form'].apply(lambda x: sum([3 if res == 'W' else 1 if res == 'D' else 0 for res in x]))
        df_teams_series['away_expected_total_points'] = df_teams_series['expected_away_form'].apply(lambda x: sum([3 if res == 'W' else 1 if res == 'D' else 0 for res in x]))
        df_teams_series['away_expected_points_diff'] = df_teams_series['away_total_points'] - df_teams_series['away_expected_total_points']
        df_teams_series['away_league_pos'] = df_teams_series['away_total_points'].rank(ascending=False, method='min')
        

        df_league_pos = df_teams_series[['team_id', 'team', 'league_pos', 'expected_points_diff', 'home_league_pos', 'home_expected_points_diff', 'away_league_pos', 'away_expected_points_diff']]

        return df_league_pos[df_league_pos['team_id'] == team_id]


def get_game_teams_series(df_teams_series: pd.DataFrame, df_game: pd.DataFrame, 
                          max_games_series: int = 5,
                          max_games_loc_series: int = 3) -> (pd.DataFrame, pd.DataFrame):
    '''
        This function takes a DataFrame of a game and returns the form of the home and away teams up to a certain matchweek.
        
        Parameters:
            - df_teams_series (pd.DataFrame): DataFrame of teams with their form
            - df_game (pd.DataFrame): DataFrame of a game
            
        Returns:
            - df_home_serie, df_away_serie (pd.DataFrame, pd.DataFrame): Tuple of dataframe with the form of the home and away teams
    '''
        
    matchweek = df_game['matchweek']
    home_team = df_game['home_team']
    home_team_id = df_game['home_team_id']
    away_team = df_game['away_team']
    away_team_id = df_game['away_team_id']

    df_home_series = df_teams_series[(df_teams_series['team_id'] == home_team_id) & (df_teams_series['team'] == home_team)]
    df_away_series = df_teams_series[(df_teams_series['team_id'] == away_team_id) & (df_teams_series['team'] == away_team)]

    home_game_before_matchweek = df_home_series['location_table'].str.slice(0, matchweek-1).str.count('H').values[0]
    home_start_serie = max(0, home_game_before_matchweek - max_games_loc_series)
    home_end_serie = home_game_before_matchweek
    dict_home_series = {
            'form': df_home_series.apply(lambda row: row['form'][max(0, matchweek - max_games_series - 1):matchweek-1], axis=1),
            'expected_form': df_home_series.apply(lambda row: row['expected_form'][max(0, matchweek - max_games_series - 1):matchweek-1], axis=1),
            'league_pos': get_table_pos(df_teams_series, matchweek, home_team_id)['league_pos'],
            'expected_points_diff': get_table_pos(df_teams_series, matchweek, home_team_id)['expected_points_diff'],
            'home_form': df_home_series.apply(lambda row: row['home_form'][home_start_serie:home_end_serie], axis=1),
            'expected_home_form': df_home_series.apply(lambda row: row['expected_home_form'][home_start_serie:home_end_serie], axis=1),
            'home_league_pos': get_table_pos(df_teams_series, matchweek, home_team_id)['home_league_pos'],
            'home_expected_points_diff': get_table_pos(df_teams_series, matchweek, home_team_id)['home_expected_points_diff'],
            'home_average_goals_form': (df_home_series['home_goals_form'].apply(
                lambda x: (sum(int(num) for num in x.split('-')[home_start_serie:home_end_serie] 
                               if num) /min(max_games_loc_series, home_game_before_matchweek))
                if min(max_games_loc_series, home_game_before_matchweek) > 0 else np.nan)),
            'home_average_xg_form': (df_home_series['home_xg_form'].apply(
                lambda x: sum(float(num) for num in x.split('-')[home_start_serie:home_end_serie] 
                              if num) / min(max_games_loc_series, home_game_before_matchweek) 
                if min(max_games_loc_series, home_game_before_matchweek) > 0 else np.nan)),
            'home_average_goals_form_against': (df_home_series['home_goals_form_against'].apply(
                lambda x: sum(int(num) for num in x.split('-')[home_start_serie:home_end_serie] 
                              if num) / min(max_games_loc_series, home_game_before_matchweek) 
                if min(max_games_loc_series, home_game_before_matchweek) > 0 else np.nan)),
            'home_average_xg_form_against': (df_home_series['home_xg_form_against'].apply(
                lambda x: sum(float(num) for num in x.split('-')[home_start_serie:home_end_serie] 
                              if num) / min(max_games_loc_series, home_game_before_matchweek) 
                if min(max_games_loc_series, home_game_before_matchweek) > 0 else np.nan)),
    }

    away_game_before_matchweek = df_away_series['location_table'].str.slice(0, matchweek-1).str.count('A').values[0]
    away_start_serie = max(0, away_game_before_matchweek - max_games_loc_series)
    away_end_serie = away_game_before_matchweek
    dict_away_series = {
            'form': df_away_series.apply(lambda row: row['form'][max(0, matchweek - max_games_series - 1):matchweek-1], axis=1),
            'expected_form': df_away_series.apply(lambda row: row['expected_form'][max(0, matchweek - max_games_series - 1):matchweek-1], axis=1),
            'league_pos': get_table_pos(df_teams_series, matchweek, away_team_id)['league_pos'],
            'expected_points_diff': get_table_pos(df_teams_series, matchweek, away_team_id)['expected_points_diff'],
            'away_form': df_away_series.apply(lambda row: row['away_form'][away_start_serie:away_end_serie], axis=1),
            'expected_away_form': df_away_series.apply(lambda row: row['expected_away_form'][away_start_serie:away_end_serie], axis=1),
            'away_league_pos': get_table_pos(df_teams_series, matchweek, away_team_id)['away_league_pos'],
            'away_expected_points_diff': get_table_pos(df_teams_series, matchweek, away_team_id)['away_expected_points_diff'],
            'away_average_goals_form': (df_away_series['away_goals_form'].apply(
                lambda x: sum(int(num) for num in x.split('-')[away_start_serie:away_end_serie] 
                              if num) / min(max_games_loc_series, away_game_before_matchweek) 
                if min(max_games_loc_series, away_game_before_matchweek) > 0 else np.nan)),
            'away_average_xg_form': (df_away_series['away_xg_form'].apply(
                lambda x: sum(float(num) for num in x.split('-')[away_start_serie:away_end_serie] 
                              if num) / min(max_games_loc_series, away_game_before_matchweek) 
                if min(max_games_loc_series, away_game_before_matchweek) > 0 else np.nan)),
            'away_average_goals_form_against': (df_away_series['away_goals_form_against'].apply(
                lambda x: sum(int(num) for num in x.split('-')[away_start_serie:away_end_serie] 
                              if num) / min(max_games_loc_series, away_game_before_matchweek) 
                if min(max_games_loc_series, away_game_before_matchweek) > 0 else np.nan)),
            'away_average_xg_form_against': (df_away_series['away_xg_form_against'].apply(
                lambda x: sum(float(num) for num in x.split('-')[away_start_serie:away_end_serie] 
                              if num) / min(max_games_loc_series, away_game_before_matchweek) 
                if min(max_games_loc_series, away_game_before_matchweek) > 0 else np.nan))
    }

    df_home_team_serie = pd.DataFrame(dict_home_series)
    df_away_team_serie = pd.DataFrame(dict_away_series)
    
    #print(f'Matchweek {matchweek} | {home_team} form: {home_serie.values[0]} - \n{away_team} form: {away_serie.values[0]}')
    return df_home_team_serie, df_away_team_serie


def get_game_players(df_game: pd.DataFrame) -> pd.DataFrame:
    '''
        This function takes a DataFrame of a game and returns a DataFrame of players for each team who participates at the game.
        
        Parameters:
            - df_game (pd.DataFrame): DataFrame of a game
            
        Returns:
            - home_players, away_players (pd.DataFrame): DataFrames of players for each team
    '''
            
    home_players_stats = df_game['home_player_stats'].apply(pd.DataFrame)
    away_players_stats = df_game['away_player_stats'].apply(pd.DataFrame)

    home_players_summary = home_players_stats['Summary'].apply(pd.Series)
    away_players_summary = away_players_stats['Summary'].apply(pd.Series)
    
    home_players_summary.columns = [col[1].strip().lower() if 'Unnamed' in col[0] else '_'.join(col).strip().lower() for col in home_players_summary.columns.values]
    home_players_summary.columns = [col.lower() for col in home_players_summary.columns]
    away_players_summary.columns = [col[1].strip() if 'Unnamed' in col[0] else '_'.join(col).strip() for col in away_players_summary.columns.values]
    away_players_summary.columns = [col.lower() for col in away_players_summary.columns]
    
    home_players = home_players_summary[~home_players_summary['player'].str.contains("Player", na=False)][['player', 'pos', 'min']]
    away_players = away_players_summary[~away_players_summary['player'].str.contains("Player", na=False)][['player', 'pos', 'min']]

    return home_players, away_players

In [ ]:
# match_europe_avant, match_europe_apres, match_europe_avant_versus, match_europe_apres_versus

df_pl_games = read_pickle('../data/epl_2024_2025.pkl')
df_pl_games.drop(columns=['link'], inplace=True)

df_teams_series = construct_teams_series(df_pl_games)
df_full_games = pd.DataFrame()

for game_index in tqdm(range(0, len(df_pl_games) -1)): 
    df_game = df_pl_games.loc[game_index]
    home_team_serie, away_team_serie = get_game_teams_series(df_teams_series, df_game)
            
    current_game_dict = {
        'date': df_game['date'],
        'league': df_game['league'],
        'matchweek': df_game['matchweek'],
        
        'home_team_id': df_game['home_team_id'],
        'home_team': df_game['home_team'],
        'home_manager': df_game['home_manager'],
        'home_formation': df_game['home_formation'],
        'home_goals': df_game['home_goals'],
        'home_xg': df_game['home_xg'],
        
        'away_xg': df_game['away_xg'],
        'away_goals': df_game['away_goals'],
        'away_team': df_game['away_team'],
        'away_manager': df_game['away_manager'],
        'away_formation': df_game['away_formation'],
        'away_team_id': df_game['away_team_id'],
        
        'home_team_form': home_team_serie['form'].values[0],
        'home_team_league_pos': home_team_serie['league_pos'].values[0],
        'home_team_points_diff': home_team_serie['expected_points_diff'].values[0],
        'home_team_home_form': home_team_serie['home_form'].values[0],
        'home_team_home_league_pos': home_team_serie['home_league_pos'].values[0],
        'home_team_home_points_diff': home_team_serie['home_expected_points_diff'].values[0],
        'home_team_average_goals_form': home_team_serie['home_average_goals_form'].values[0],
        'home_team_average_xg_form': home_team_serie['home_average_xg_form'].values[0],
        'home_team_average_goals_form_against': home_team_serie['home_average_goals_form_against'].values[0],
        'home_team_average_xg_form_against': home_team_serie['home_average_xg_form_against'].values[0],
        
        'away_team_form': away_team_serie['form'].values[0],
        'away_team_league_pos': away_team_serie['league_pos'].values[0],
        'away_team_points_diff': away_team_serie['expected_points_diff'].values[0],
        'away_team_away_form': away_team_serie['away_form'].values[0],
        'away_team_away_league_pos': away_team_serie['away_league_pos'].values[0],
        'away_team_away_points_diff': away_team_serie['away_expected_points_diff'].values[0],
        'away_team_average_goals_form': away_team_serie['away_average_goals_form'].values[0],
        'away_team_average_xg_form': away_team_serie['away_average_xg_form'].values[0],
        'away_team_average_goals_form_against': away_team_serie['away_average_goals_form_against'].values[0],
        'away_team_average_xg_form_against': away_team_serie['away_average_xg_form_against'].values[0],
    }
        
    df_current_game = pd.DataFrame([current_game_dict])
    df_full_games = pd.concat([df_full_games, df_current_game]).reset_index(drop=True)

df_full_games.to_csv('../data/epl_2024_2025_full.csv', index=False)

  0%|          | 0/109 [00:00<?, ?it/s]

In [ ]:
# Player -> Ne sert pas 
df_pl_games = read_pickle('../data/epl_2024_2025.pkl')
df_teams_series = construct_teams_series(df_pl_games)
df_players = pd.DataFrame()

for game_index in tqdm(range(0, len(df_pl_games) - 1)):
    home_team_serie, away_team_serie = get_game_teams_series(df_teams_series, df_pl_games.loc[game_index])
    home_players, away_players = get_game_players(df_pl_games.loc[game_index])
    
    home_players['matchweek'] = df_pl_games.loc[game_index]['matchweek']
    home_players['team_id'] = df_pl_games.loc[game_index]['home_team_id']
    home_players['team'] = df_pl_games.loc[game_index]['home_team']
    home_players['team_form'] = home_team_serie
    home_players['location'] = 'home'
    home_players['versus_team_id'] = df_pl_games.loc[game_index]['away_team_id']
    home_players['versus_team'] = df_pl_games.loc[game_index]['away_team']
    home_players['versus_team_form'] = away_team_serie
    
    away_players['matchweek'] = df_pl_games.loc[game_index]['matchweek']
    away_players['team_id'] = df_pl_games.loc[game_index]['away_team_id']
    away_players['team'] = df_pl_games.loc[game_index]['away_team']
    away_players['location'] = 'away'
    away_players['team_form'] = away_team_serie
    away_players['versus_team_id'] = df_pl_games.loc[game_index]['home_team_id']
    away_players['versus_team'] = df_pl_games.loc[game_index]['home_team']
    away_players['versus_team_form'] = home_team_serie
    
    df_players = pd.concat([df_players, home_players, away_players])

df_players.head(5)
        

'WW'